In [1]:
suits = ("Clubs", "Diamonds", "Hearts", "Spades") 
ranks = {
            "Two": 2, "Three" : 3, "Four" : 4, "Five" : 5, "Six" : 6, "Seven" : 7, "Eight" : 8, "Nine" : 9, "Ten" : 10,
            "Jack" : 10, "Queen" : 10, "King" : 10, "Ace" : 11
        }
suits_unicodes = {"Clubs": "\u2663", "Diamonds" : "\u2666", "Hearts" : "\u2665", "Spades": "\u2660"}

In [2]:
from colorama import Fore, Style

class Card():
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        
    def __str__(self):
        unicode = suits_unicodes[self.suit]
        rank = get_rank_to_print(self.suit, self.rank, ranks[self.rank])
        if self.suit == "Clubs" or self.suit == "Spades":
            return generate_card(rank, unicode, Fore.BLACK)
        else:
            return generate_card(rank, unicode, Fore.RED)
         

In [3]:
def get_rank_to_print(suit, rank, value):
        if rank == "Jack":
            return "J"
        elif rank == "Queen":
            return "Q"
        elif rank == "King":
            return "K" 
        elif rank == "Ace":
            return "A" 
        else:
            return f"{value}"
        
def generate_card(rank, unicode, color):
    if rank == "10" :
        return f'''{color}
            ┌────────┐
            | {rank}     |
            |        |
            |   {unicode}    |
            |        |
            |    {rank}  |
            └────────┘
            {Style.RESET_ALL}'''
    else:
        return f'''{color}
            ┌───────┐
            | {rank}     |
            |       |
            |   {unicode}   |
            |       |
            |    {rank}  |
            └───────┘
            {Style.RESET_ALL}'''


In [4]:
import random

class Deck():
    
    def __init__(self, suits, ranks):
        self.cards = []
        for suit in suits:
            for rank in ranks.keys():
                self.cards.append(Card(suit, rank))
                    
    def shuffle(self):
        random.shuffle(self.cards)
    
    def deal(self):
        card = self.cards.pop()
        return card
    
    def __str__(self):
        result = []
        for card in self.cards:
            result.append(str(card))
        return "\n".join(result)

In [5]:
class Hand():
    
    def __init__(self):
        self.cards = []
        self.aces = 0
        self.value = 0
    
    def add_card(self, card):
        self.cards.append(card) 
        self.value += ranks[card.rank]
        if card.rank == "Ace":
            self.aces += 1 
            
    def adjust_aces(self):
        while self.value > 21 and self.aces > 0:
            self.value -= 10
            self.aces -= 1
    
    def show_cards_partial(self):
        print(self.cards[0])
    
    def show_cards_full(self): 
        print(self)
            
    def __str__(self):
        result = []
        for card in self.cards:
            result.append(str(card))
        cards_str = " ".join(result)
        return cards_str   

In [6]:
class Chips():
    
    def __init__(self, total):
        self.total = total
        self.bet = 0
    
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet
        

In [7]:
def ask_user_for_total_chips(max_chips): 
    while True:
        try:
            total_chips = int(input(f"Enter how many chips you want to play with [Max: {max_chips}]: \n"))
        except:
            print(f"Number of chips should be between 1 and {max_chips}\n")
            continue
        else:
            if total_chips > max_chips:
                print(f"Number of chips should be between 1 and {max_chips}\n")
                continue
            else:
                return total_chips            

In [8]:
def ask_user_bet(chips):
    while True:
        try:
            chips.bet = int(input(f"Enter bets between 1 and {chips.total}: \n"))
        except:
            print(f"Invalid bet! Enter bets between 1 and {chips.total}.\n")
        else:
            if 1 <= chips.bet <= chips.total:
                break
            else:
                print(f"Invalid bet! Enter bets between 1 and {chips.total}.\n")
            

In [9]:
import time


def play_blackjack():
    print("Welcome to Blackjack.\n")
    
    max_chips = 500
    total_chips = ask_user_for_total_chips(max_chips)
    player_chips = Chips(total_chips)    
        
    while True:
        player_playing = True
        # Create deck
        deck = Deck(suits, ranks)
        deck.shuffle() 
        
        # Setup player chips  
        ask_user_bet(player_chips)
        
        # Setup dealer hand
        dealer_hand = Hand()
        
        # Setup player hand
        player_hand = Hand()

  
        # Deal cards
        player_hand.add_card(deck.deal())
        dealer_hand.add_card(deck.deal())
        player_hand.add_card(deck.deal())
        dealer_hand.add_card(deck.deal())
        
        print("----Dealer----")
        dealer_hand.show_cards_partial()
        print("----Player----")
        player_hand.show_cards_full() 
        
        # Prompt player hit or stand
        
        while player_playing:
            try:
                hit_or_stand = input("Hit(h) or Stand(s): ").lower()[0]
            except:
                print("Invalid value! Enter Hit(h) or Stand(s).")
            else:
                print("\n")
                if hit_or_stand == "h":
                    # Hit
                    card = deck.deal()
                    player_hand.add_card(card)
                    player_hand.adjust_aces()
                    if player_hand.value > 21:
                        # Bust
                        player_playing = False
                        print("Player Bust!")
                        print(card)
                        break
                    else:
                        print("----Dealer----")
                        dealer_hand.show_cards_partial()
                        print("----Player----")
                        player_hand.show_cards_full() 
                        continue
                else:
                    time.sleep(1) 
                    break
        
        if player_hand.value < 21:
            
            while dealer_hand.value < 17: 
                print("----Dealer----")
                dealer_hand.add_card(deck.deal())
                dealer_hand.adjust_aces()
                dealer_hand.show_cards_full()
                print("----Player----")
                player_hand.show_cards_full() 
                time.sleep(1)
            
            if dealer_hand.value > 21: 
                print("--------------")
                print("Dealer bust!")
                print("--------------")
                player_chips.win_bet()
            elif dealer_hand.value > player_hand.value: 
                print("--------------")
                print("Dealer wins!")
                print("--------------")
                player_chips.lose_bet()
            elif player_hand.value > dealer_hand.value: 
                print("--------------")
                print("Player wins!")
                print("--------------")
                player_chips.win_bet()
            else:
                print("--------------")
                print("It's a tie!")
                print("--------------")
  

        print(f"\nRemaining chips: {player_chips.total}\n")
        
          # Ask to play again
        if total_chips > 0:
            new_game = input("Play another game? Enter Yes[y] or No[n].\n")
            if new_game.lower()[0] == 'y':
                player_playing=True
                continue
            else:
                print("Thanks for playing!")
                break
        else:
            print("Not enough chips! Thanks for playing.")
            break
    
       
            

In [ ]:
play_blackjack()

Welcome to Blackjack.



Enter how many chips you want to play with [Max: 500]: 
 200
Enter bets between 1 and 200: 
 1


----Dealer----

            ┌───────┐
            | 4     |
            |       |
            |   ♦   |
            |       |
            |    4  |
            └───────┘
            
----Player----

            ┌───────┐
            | J     |
            |       |
            |   ♠   |
            |       |
            |    J  |
            └───────┘
             
            ┌───────┐
            | 4     |
            |       |
            |   ♠   |
            |       |
            |    4  |
            └───────┘
            


Hit(h) or Stand(s):  h




----Dealer----

            ┌───────┐
            | 4     |
            |       |
            |   ♦   |
            |       |
            |    4  |
            └───────┘
            
----Player----

            ┌───────┐
            | J     |
            |       |
            |   ♠   |
            |       |
            |    J  |
            └───────┘
             
            ┌───────┐
            | 4     |
            |       |
            |   ♠   |
            |       |
            |    4  |
            └───────┘
             
            ┌───────┐
            | A     |
            |       |
            |   ♦   |
            |       |
            |    A  |
            └───────┘
            


Hit(h) or Stand(s):  h




Player Bust!

            ┌───────┐
            | 8     |
            |       |
            |   ♥   |
            |       |
            |    8  |
            └───────┘
            

Remaining chips: 200

